In [6]:
import cv2
import glob
import os

def load_points_from_file(file_path):
    """
    指定のtxtファイルから (frame, track, x, y, attr) のタプルを読み込みます。
    """
    points = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(',')
            if len(parts) < 5:
                continue
            frame = int(parts[0])
            track = int(parts[1])
            x = float(parts[2])
            y = float(parts[3])
            attr = parts[4]
            points.append((frame, track, x, y, attr))
    return points

def scale_points_swapped(points, img_width, draw_height):
    """
    各点のx, y座標を反転させます。
    
    - x座標は0～img_widthの範囲で反転させます（例: 0ならimg_width、img_widthなら0）。
    - y座標は0～draw_heightの範囲で反転させます（例: 0ならdraw_height、draw_heightなら0）。
    """
    flipped_points = []
    for point in points:
        frame, track, x, y, attr = point
        flipped_x = img_width - x
        flipped_points.append((frame, track, flipped_x, y, attr))
    return flipped_points


def get_color(attr):
    """
    属性に応じた色を返します。
    属性が 'O' で始まるならピンク、'D' で始まるなら紫を返します。
    """
    if attr.startswith('O'):
        # green
        return (0, 200, 0)
        
        # pink
        # return (180, 105, 255)
        
        # yellow
        # return (0, 255, 247)

        # purple
        # return (128, 0, 128)
    elif attr.startswith('D'):
        # green
        # return (0, 200, 0)
        
        # pink
        return (180, 105, 255)
        
        # yellow
        # return (0, 255, 247)
    
        # purple
        # return (128, 0, 128)
    else:
        return (0, 0, 0)

def build_attribute_color_mapping_from_file(gt_file):
    """
    指定のground_truth側のtxtファイルから、属性(attr)→色のマッピング辞書を作成します。  
    同じ属性が出現した場合、最初に出現した属性の色を採用します。
    """
    mapping = {}
    pts = load_points_from_file(gt_file)
    for p in pts:
        _, _, _, _, attr = p
        if attr not in mapping:
            mapping[attr] = get_color(attr)
    return mapping

def draw_points(image, points):
    """
    画像上に各点を大きめの円と拡大された属性テキストで描画します。
    """
    for point in points:
        frame, track, x, y, attr = point
        color = get_color(attr)
        # 座標をintに変換
        x_int, y_int = int(x), int(y)
        # 円の半径を25に、塗りつぶし(-1)
        cv2.circle(image, (x_int, y_int), radius=25, color=color, thickness=-1)
        # テキストの描画位置も整数に変換
        cv2.putText(image, attr, (x_int + 10, y_int - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 2.5, color, thickness=6)
    return image


def draw_points_with_mapping(image, points, attr_color_mapping):
    """
    画像上に各点を描画します。  
    各点の属性(attr)が attr_color_mapping に存在する場合は、その色で描画し、
    なければ get_color(attr) による色で描画します。  
    円の半径は20、テキストはフォントスケール1.5、太さ3で描画します。
    """
    for point in points:
        frame, track, x, y, attr = point
        if attr in attr_color_mapping:
            color = attr_color_mapping[attr]
        else:
            color = get_color(attr)
        # 座標をintに変換
        x_int, y_int = int(x), int(y)
        # 円の半径を25に、塗りつぶし(-1)
        cv2.circle(image, (x_int, y_int), radius=25, color=color, thickness=-1)
        # テキストの描画位置も整数に変換
        cv2.putText(image, attr, (x_int + 10, y_int - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 2.5, color, thickness=6)
    return image

In [ ]:
# 共通設定
base_name = 'IMG_0112_3'
# 背景画像の読み込みとリサイズ（縦1105×横1505）
base_image_path = '../../court_images/Outdoor.png'
base_image = cv2.imread(base_image_path)
if base_image is None:
    raise FileNotFoundError(f"背景画像の読み込みに失敗しました: {base_image_path}")
resized_image = cv2.resize(base_image, (1505, 1105))
height, width = resized_image.shape[:2]
print(f"画像サイズ: width={width}, height={height}, 描画領域: {height}px")

# 出力フォルダ
output_dir = '../../videos/Outdoor/top/minimap'
os.makedirs(output_dir, exist_ok=True)

gt_txt = os.path.join('../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball', f'{base_name}.txt')
if not os.path.isfile(gt_txt):
    print(f"GTファイルが見つかりません: {gt_txt}")
else:
    pts = load_points_from_file(gt_txt)
    if pts:
        scaled = scale_points_swapped(pts, width, height)
        frames = {}
        for f, *data in scaled:
            frames.setdefault(f, []).append([f, *data])
        # FPS取得
        orig_gt = os.path.join('../../videos/Outdoor/top/split', f'{base_name}.MOV')
        cap = cv2.VideoCapture(orig_gt)
        fps = cap.get(cv2.CAP_PROP_FPS) or 10
        cap.release()
        print(f"GT Using FPS: {fps}")
        # 動画書き出し
        vw = cv2.VideoWriter(os.path.join(output_dir, f'{base_name}_GT.mp4'),
                             cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        for f in sorted(frames):
            img = resized_image.copy()
            img = draw_points(img, frames[f])
            vw.write(img)
        vw.release()
        print(f"GT動画保存: {base_name}_GT.mp4")

pred_txt_pattern = os.path.join('../../output/CAMELTrack_outputs/Outdoor/transformed/with_jersey_number/with_team', f'{base_name}.txt')
txts = glob.glob(pred_txt_pattern)
if not txts:
    print(f"Predファイルが見つかりません: {pred_txt_pattern}")
else:
    for pred_txt in txts:
        print(f"Processing Pred: {pred_txt}")
        pts = load_points_from_file(pred_txt)
        if not pts:
            continue
        scaled = scale_points_swapped(pts, width, height)
        frames = {}
        for f, *data in scaled:
            frames.setdefault(f, []).append([f, *data])
        # 属性マッピング
        gt_txt = os.path.join('../../ground_truth/Outdoor/MOT_files/split_transformed/check_ball', f'{base_name}.txt')
        mapping = {}  
        if os.path.isfile(gt_txt):
            mapping = build_attribute_color_mapping_from_file(gt_txt)
        # FPS取得
        orig_pred = os.path.join('../../videos/Outdoor/top/split', f'{base_name}.MOV')
        cap = cv2.VideoCapture(orig_pred)
        fps = cap.get(cv2.CAP_PROP_FPS) or 10
        cap.release()
        print(f"Pred Using FPS: {fps}")
        # 動画書き出し
        vw = cv2.VideoWriter(os.path.join(output_dir, f'{base_name}_pred.mp4'),
                             cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        for f in sorted(frames):
            img = resized_image.copy()
            img = draw_points_with_mapping(img, frames[f], mapping)
            vw.write(img)
        vw.release()
        print(f"Pred動画保存: {base_name}_pred.mp4")

画像サイズ: width=1505, height=1105, 描画領域: 1105px
GT Using FPS: 29.97002997002997
GT動画保存: IMG_0112_3_GT.mp4
Processing Pred: ../../CAMELTrack_outputs/Outdoor/transformed/with_jersey_number/with_team/IMG_0112_3.txt
Pred Using FPS: 29.97002997002997
Pred動画保存: IMG_0112_3_pred.mp4
